# Configuration de Cuda dans Google Colab

In [1]:
!nvcc -V  

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [2]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-wr0izjtj
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-wr0izjtj
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4305 sha256=33d07f74a3959244d9ecf6ecfb400f6b3112d6d4fdbbb6702718cb9b9ac9a247
  Stored in directory: /tmp/pip-ephem-wheel-cache-f36ihvnr/wheels/db/c1/1f/a2bb07bbb4a1ce3c43921252aeafaa6205f08637e292496f04
Successfully built NVCCPlugin


## On vérifie que l'on est bien connecté au GPU

In [3]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri Mar 31 12:40:01 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Chargement du plugin nvcc permettant de compiler/executer les programmes Cuda

In [4]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


## Un makefile est déjà à votre disposition pour compiler les programme du TP


### 1. Executez la cellule du Makefile

Le makefile a été modifié pour les programmes puisse s'exécuter avec la GPU premium.



In [5]:
%%writefile Makefile
# Change the example variable to build a different source module (e.g. EXAMPLE=exercice01)
EXAMPLE=program

# Makefile variables 
# Add extra targets to OBJ with space separator e.g. If there is as source file random.c then add random.o to OBJ)
# Add any additional dependancies (header files) to DEPS. e.g. if there is aheader file random.h required by your source modules then add this to DEPS.
CC=gcc
CFLAGS= -O3 -Wextra -fopenmp
NVCC=nvcc
NVCC_FLAGS= -gencode arch=compute_75,code=sm_75 -gencode arch=compute_80,code=sm_80 -gencode arch=compute_86,code=sm_86 -gencode arch=compute_87,code=sm_87
OBJ=$(EXAMPLE).o
DEPS=

# Build rule for object files ($@ is left hand side of rule, $< is first item from the right hand side of rule)
%.o : %.cu $(DEPS)
	$(NVCC) -c -o $@ $< $(NVCC_FLAGS) $(addprefix -Xcompiler ,$(CCFLAGS))

# Make example ($^ is all items from right hand side of the rule)
$(EXAMPLE) : $(OBJ)
	$(NVCC) -o $@ $^ $(NVCC_FLAGS) $(addprefix -Xcompiler ,$(CCFLAGS))

# PHONY prevents make from doing something with a filename called clean
.PHONY : clean
clean:
	rm -rf $(EXAMPLE) $(OBJ)

Writing Makefile


# TP5



In [6]:
%%writefile utils.h
#ifndef __UTILS_H__
#define __UTILS_H__
#include <stdio.h>

static void HandleError( cudaError_t err,
                         const char *file,
                         int line ) {
    if (err != cudaSuccess) {
        printf( "%s in %s at line %d\n", cudaGetErrorString( err ),
                file, line );
        exit( EXIT_FAILURE );
    }
}
#define HANDLE_ERROR( err ) (HandleError( err, __FILE__, __LINE__ ))


#define HANDLE_NULL( a ) {if (a == NULL) { \
                            printf( "Host memory failed in %s at line %d\n", \
                                    __FILE__, __LINE__ ); \
                            exit( EXIT_FAILURE );}}

template< typename T >
void swap( T& a, T& b ) {
    T t = a;
    a = b;
    b = t;
}


void* big_random_block( int size ) {
    unsigned char *data = (unsigned char*)malloc( size );
    HANDLE_NULL( data );
    for (int i=0; i<size; i++)
        data[i] = rand();

    return data;
}

int* big_random_block_int( int size ) {
    int *data = (int*)malloc( size * sizeof(int) );
    HANDLE_NULL( data );
    for (int i=0; i<size; i++)
        data[i] = rand();

    return data;
}


// a place for common kernels - starts here

__device__ unsigned char value( float n1, float n2, int hue ) {
    if (hue > 360)      hue -= 360;
    else if (hue < 0)   hue += 360;

    if (hue < 60)
        return (unsigned char)(255 * (n1 + (n2-n1)*hue/60));
    if (hue < 180)
        return (unsigned char)(255 * n2);
    if (hue < 240)
        return (unsigned char)(255 * (n1 + (n2-n1)*(240-hue)/60));
    return (unsigned char)(255 * n1);
}

__global__ void float_to_color( unsigned char *optr,
                              const float *outSrc ) {
    // map from threadIdx/BlockIdx to pixel position
    int x = threadIdx.x + blockIdx.x * blockDim.x;
    int y = threadIdx.y + blockIdx.y * blockDim.y;
    int offset = x + y * blockDim.x * gridDim.x;

    float l = outSrc[offset];
    float s = 1;
    int h = (180 + (int)(360.0f * outSrc[offset])) % 360;
    float m1, m2;

    if (l <= 0.5f)
        m2 = l * (1 + s);
    else
        m2 = l + s - l * s;
    m1 = 2 * l - m2;

    optr[offset*4 + 0] = value( m1, m2, h+120 );
    optr[offset*4 + 1] = value( m1, m2, h );
    optr[offset*4 + 2] = value( m1, m2, h -120 );
    optr[offset*4 + 3] = 255;
}

__global__ void float_to_color( uchar4 *optr,
                              const float *outSrc ) {
    // map from threadIdx/BlockIdx to pixel position
    int x = threadIdx.x + blockIdx.x * blockDim.x;
    int y = threadIdx.y + blockIdx.y * blockDim.y;
    int offset = x + y * blockDim.x * gridDim.x;

    float l = outSrc[offset];
    float s = 1;
    int h = (180 + (int)(360.0f * outSrc[offset])) % 360;
    float m1, m2;

    if (l <= 0.5f)
        m2 = l * (1 + s);
    else
        m2 = l + s - l * s;
    m1 = 2 * l - m2;

    optr[offset].x = value( m1, m2, h+120 );
    optr[offset].y = value( m1, m2, h );
    optr[offset].z = value( m1, m2, h -120 );
    optr[offset].w = 255;
}


#if _WIN32
    //Windows threads.
    #include <windows.h>

    typedef HANDLE CUTThread;
    typedef unsigned (WINAPI *CUT_THREADROUTINE)(void *);

    #define CUT_THREADPROC unsigned WINAPI
    #define  CUT_THREADEND return 0

#else
    //POSIX threads.
    #include <pthread.h>

    typedef pthread_t CUTThread;
    typedef void *(*CUT_THREADROUTINE)(void *);

    #define CUT_THREADPROC void
    #define  CUT_THREADEND
#endif

//Create thread.
CUTThread start_thread( CUT_THREADROUTINE, void *data );

//Wait for thread to finish.
void end_thread( CUTThread thread );

//Destroy thread.
void destroy_thread( CUTThread thread );

//Wait for multiple threads.
void wait_for_threads( const CUTThread *threads, int num );

#if _WIN32
    //Create thread
    CUTThread start_thread(CUT_THREADROUTINE func, void *data){
        return CreateThread(NULL, 0, (LPTHREAD_START_ROUTINE)func, data, 0, NULL);
    }

    //Wait for thread to finish
    void end_thread(CUTThread thread){
        WaitForSingleObject(thread, INFINITE);
        CloseHandle(thread);
    }

    //Destroy thread
    void destroy_thread( CUTThread thread ){
        TerminateThread(thread, 0);
        CloseHandle(thread);
    }

    //Wait for multiple threads
    void wait_for_threads(const CUTThread * threads, int num){
        WaitForMultipleObjects(num, threads, true, INFINITE);

        for(int i = 0; i < num; i++)
            CloseHandle(threads[i]);
    }

#else
    //Create thread
    CUTThread start_thread(CUT_THREADROUTINE func, void * data){
        pthread_t thread;
        pthread_create(&thread, NULL, func, data);
        return thread;
    }

    //Wait for thread to finish
    void end_thread(CUTThread thread){
        pthread_join(thread, NULL);
    }

    //Destroy thread
    void destroy_thread( CUTThread thread ){
        pthread_cancel(thread);
    }

    //Wait for multiple threads
    void wait_for_threads(const CUTThread * threads, int num){
        for(int i = 0; i < num; i++)
            end_thread( threads[i] );
    }

#endif

#endif  // __UTILS_H__

Writing utils.h


## 1. Reduction via produit scalaire

### Implémentez le kernel : 

#### 1.1 Creez le vecteur **cache** de type float et de taille **threadsPerBlock**;
#### 1.2 Trouver l'index global des threads
#### 1.3 Comme pour l'addition vectoriel, multipliez les éléments d'index tid des vecteurs a et b entre eux. Puis stockez le résultat dans le cache.
#### 1.4 implémentez la reduction comme énoncé dans github.

#### 1.5 Completez le main afin de pouvoir exécuter le programme
- Allocation mémoire
- Copie mémoire H2D
- Lancement du kernel
- Copie mémoire D2H
- Liberation de la mémoire

In [7]:
%%writefile program.cu
#include "utils.h"


const int N = 33792;
const int threadsPerBlock = 256;
const int blocksPerGrid = (N+threadsPerBlock-1) / threadsPerBlock;


__global__ void produit_scalaire( float *a, float *b, float *c ) {
    // 1.1 Creation du vecteur cache dans la mémoire partagée.
    __shared__ float cache[threadsPerBlock];

    // 1.2 Trouver l'index global de chaque thread (on est en 1D) et l'index local cacheIndex dans la mémoire partagée.
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    int cacheIndex = threadIdx.x;

    // 1.3 Comme pour l'addition vectoriel, multipliez les éléments d'index tid des vecteurs a et b entre eux. Puis stockez le résultat dans le cache.
    float temp = 0;
    while (tid < N) {
        temp += a[tid] * b[tid];
        tid += blockDim.x * gridDim.x;
    }
    cache[cacheIndex] = temp;

    //1.4 implémentez la reduction comme énoncé dans github.
    __syncthreads();

    int i = blockDim.x/2;
    while (i != 0) {
        if (cacheIndex < i) {
            cache[cacheIndex] += cache[cacheIndex + i];
        }
        __syncthreads();
        i /= 2;
    }

    if (cacheIndex == 0) {
        c[blockIdx.x] = cache[0];
    }
}


int main( void ) {
    float   *a, *b, c, *partial_c;
    float   *dev_a, *dev_b, *dev_partial_c;

    a = (float*)malloc( N*sizeof(float) );
    b = (float*)malloc( N*sizeof(float) );
    partial_c = (float*)malloc( blocksPerGrid*sizeof(float) );

    for (int i=0; i<N; i++) {
      a[i] = i;
      b[i] = i*2;
    }

    // Allocation memoire sur le GPU
    cudaMalloc(&dev_a, N * sizeof(float));
    cudaMalloc(&dev_b, N * sizeof(float));
    cudaMalloc(&dev_partial_c, blocksPerGrid * sizeof(float));

    // Copie des donnees de l'hote vers le GPU
    cudaMemcpy(dev_a, a, N * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(dev_b, b, N * sizeof(float), cudaMemcpyHostToDevice);

    // Lancement du kernel avec blocksPerGrid blocs et threadsPerBlock threads par bloc
    produit_scalaire<<<blocksPerGrid, threadsPerBlock>>>(dev_a, dev_b, dev_partial_c);

    // Copie du résultat partiel du GPU vers l'hote
    cudaMemcpy(partial_c, dev_partial_c, blocksPerGrid * sizeof(float), cudaMemcpyDeviceToHost);

    // Reduction des résultats partiels sur le CPU
    c = 0;
    for (int i=0; i<blocksPerGrid; i++) {
      c += partial_c[i];
    }

    #define sum_squares(x)  (x*(x+1)*(2*x+1)/6)
    printf( "Does GPU value %.6g = %.6g?\n", c,
         2 * sum_squares( (float)(N - 1) ) );

    // Liberation de la memoire sur le CPU et le GPU
    free( a );
    free( b );
    free( partial_c );

    cudaFree(dev_a);
    cudaFree(dev_b);
    cudaFree(dev_partial_c);
}

Writing program.cu


In [8]:
!make

nvcc -c -o program.o program.cu -gencode arch=compute_75,code=sm_75 -gencode arch=compute_80,code=sm_80 -gencode arch=compute_86,code=sm_86 -gencode arch=compute_87,code=sm_87 
nvcc -o program program.o -gencode arch=compute_75,code=sm_75 -gencode arch=compute_80,code=sm_80 -gencode arch=compute_86,code=sm_86 -gencode arch=compute_87,code=sm_87 


In [9]:
!./program

Does GPU value 2.57236e+13 = 2.57236e+13?


## 2. Multiplication matricielle 3D

### Version 1 : Mémoire globale

Completez le kernel matrixMul3D 

#### 2.1 Trouver le bon index des threads pour les colonnes (col) les lignes (row) et la profondeur (depth).

#### 2.2 Implémentez le produit scalaire
Rappel : Chaque thread effectue le produit scalaire entre une ligne de **a** et une colonne de **b**

Chaque thread itere donc sur les éléments d'une ligne de a (qui est de même taille qu'une colonne de b), le multiplie avec l'élément correspondant de la colonne de b, puis ajoute le resultat dans un registe **sum**.

A la fin de la boucle, sum contient le resultat du produit scalaire que l'on vient mettre dans **c**.

#### 2.3 Complétez la fonction main afin de pouvoir lancer le programme avec le kernel MatrixMul3D.
- Allocation mémoire
- Copie mémoire H2D
- Lancement du kernel
- Copie mémoire D2H
- Liberation de la mémoire

### Version 2 : Mémoire partagée.

#### 2.4 Creez les matrices 3D shared_a et shared_b dans la mémoire partagée, de type int et de taille BLOCK_SIZExBLOCK_SIZExBLOCK_SIZE.

#### 2.5 Trouver le bon index des threads pour les colonnes (col) les lignes (row) et la profondeur (depth).

#### 2.6 Implémentez le produit matriciel dans la mémoire partagée.
Cette fois ci, on ne calcule pas le produit matriciel au niveau du thread mais au niveau d'un bloc de threads.

Dans cet exercice, un bloc est de taille 2x2x2. Il y aura donc, pour un bloc, 8 threads qui vont calculer le produit matriciel entre une ligne de shared_a et une colonne de shared_b.

Cependant, shared_a et shared_b sont de taille 2x2x2 et non pas de taille nxnxn. Il faut donc pour chaque thread iterer le nombre de blocs en x d'une grille (gridDim.x), décaler shared_a vers la droite (de façon horizontal) en utilisant un offset (**h_stride**), et décaler shared_b vers le bas (de façon vertical en utilisant un offset (**v_stride**). Voir Figure sur github.

Pensez a synchroniser les threads lorsque vous modifiez/récupérez des valeurs dans la mémoire partagée.


#### 2.7 Executez le code en remplaçant matrixMul3D par shared_matrixMul3D
Vous devez obtenir le même resultat pour les dexu versions.

In [180]:
%%writefile program.cu
#include <stdio.h>

#define BLOCK_SIZE 2
__global__ void matrixMul3D(int *a, int *b, int *c, int n)
{
    
    // 2.1 Indexation des threads
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int depth = threadIdx.z;

    float sum = 0.0f;

    // 2.2 Produit scalaire
    if (depth < n && row < n && col < n) {

        for (int k = 0; k < n; k++) {
            //sum += a[depth * n * n + row * n + k] * b[k * n * n + col * n + depth];
            sum += a[depth * n * n + row * n + k] * b[k * n + col];        }
        c[depth * n * n + row * n + col] = sum;
    }
}

__global__ void shared_matrixMul3D(int *a, int *b, int *c, int n)
{
    // 2.4 Creez les matrices 3D shared_a et shared_b dans la mémoire partagée, de type int et de taille BLOCK_SIZE*BLOCK_SIZE*BLOCK_SIZE.
    __shared__ int shared_a[BLOCK_SIZE][BLOCK_SIZE][BLOCK_SIZE];
    __shared__ int shared_b[BLOCK_SIZE][BLOCK_SIZE][BLOCK_SIZE];

    // 2.5 Trouver le bon index des threads pour les colonnes (col) les lignes (row) et la profondeur (depth).
    int col = threadIdx.x + blockIdx.x * blockDim.x;
    int row = threadIdx.y + blockIdx.y * blockDim.y;
    int depth = threadIdx.z + blockIdx.z * blockDim.z;

    int sum = 0;
    // 2.6 Implémentez le produit matriciel dans la mémoire partagée.
    for (int m = 0; m < gridDim.x; m++) {

        int h_stride = m * BLOCK_SIZE;
        int v_stride = m * BLOCK_SIZE  ;
        shared_a[threadIdx.z][threadIdx.y][threadIdx.x] = a[depth * n * n + row * n + h_stride + threadIdx.x];
        shared_b[threadIdx.z][threadIdx.y][threadIdx.x] = b[(v_stride + threadIdx.y) * n + col + threadIdx.x];

        __syncthreads();

        for (int i = 0; i < BLOCK_SIZE; i++) {
          if((depth * n * n + row * n + col) == 5){
                printf("valeur de x : %i , valeur de y : %i \n", shared_a[threadIdx.z][threadIdx.y][i], shared_b[threadIdx.z][i][threadIdx.x]);
          }
            sum += shared_a[threadIdx.z][threadIdx.y][i] * shared_b[threadIdx.z][i][threadIdx.x];
        }

        __syncthreads();
    }


    c[depth * n * n + row * n + col] = sum;
}

void print_matrix(int*a, int n){
  for (int i = 0; i < n; i++) {
        for (int j = 0; j < n; j++) {
            for (int k = 0; k < n; k++) {
                printf("%d\t", a[i*n*n + j * n + k]);
            }
        printf("\n");
        }
        printf("\n");
    }
}

int main()
{
    int n = 4;
    int size = n * n * n * sizeof(int);
    int *h_a = (int*)malloc(size);
    int *h_b = (int*)malloc(size);
    int *h_c = (int*)malloc(size);

    for (int i = 0; i < n * n * n; ++i) {
        h_a[i] = i;
        h_b[i] = i*2;
    }
    printf("Matrix a :\n");
    print_matrix(h_a, n);
    printf("Matrix b :\n");
    print_matrix(h_b, n);
    int *d_a, *d_b, *d_c;

    // 2.3 Completez la fonction main.

    cudaMalloc((void **)&d_a, size);
    cudaMalloc((void **)&d_b, size);
    cudaMalloc((void **)&d_c, size);

    cudaMemcpy(d_a, h_a, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, h_b, size, cudaMemcpyHostToDevice);

    ////// SHARED VERSION ///////

    dim3 threadsPerBlock(BLOCK_SIZE, BLOCK_SIZE, BLOCK_SIZE);
    dim3 blocksPerGrid((n + threadsPerBlock.x - 1) / threadsPerBlock.x,
                       (n + threadsPerBlock.y - 1) / threadsPerBlock.y,
                       (n + threadsPerBlock.z - 1) / threadsPerBlock.z);

    shared_matrixMul3D<<<blocksPerGrid, threadsPerBlock>>>(d_a, d_b, d_c, n);


    cudaMemcpy(h_c, d_c, size, cudaMemcpyDeviceToHost); 

    printf("Matrix c :\n");
    print_matrix(h_c, n);

    ////// NORMAL VERSION ///////

    dim3 threadsPerBlock2(BLOCK_SIZE * 2, BLOCK_SIZE * 2, BLOCK_SIZE * 2);
    dim3 blocksPerGrid2(BLOCK_SIZE, BLOCK_SIZE,  BLOCK_SIZE);

    matrixMul3D<<<blocksPerGrid2, threadsPerBlock2>>>(d_a, d_b, d_c, n);

    cudaMemcpy(h_c, d_c, size, cudaMemcpyDeviceToHost); 

    printf("Matrix c :\n");
    print_matrix(h_c, n);

    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);
    free(h_a);
    free(h_b);
    free(h_c);
    return 0;
}

Overwriting program.cu


In [181]:
!make

nvcc -c -o program.o program.cu -gencode arch=compute_75,code=sm_75 -gencode arch=compute_80,code=sm_80 -gencode arch=compute_86,code=sm_86 -gencode arch=compute_87,code=sm_87 
nvcc -o program program.o -gencode arch=compute_75,code=sm_75 -gencode arch=compute_80,code=sm_80 -gencode arch=compute_86,code=sm_86 -gencode arch=compute_87,code=sm_87 


In [182]:
!./program

Matrix a :
0	1	2	3	
4	5	6	7	
8	9	10	11	
12	13	14	15	

16	17	18	19	
20	21	22	23	
24	25	26	27	
28	29	30	31	

32	33	34	35	
36	37	38	39	
40	41	42	43	
44	45	46	47	

48	49	50	51	
52	53	54	55	
56	57	58	59	
60	61	62	63	

Matrix b :
0	2	4	6	
8	10	12	14	
16	18	20	22	
24	26	28	30	

32	34	36	38	
40	42	44	46	
48	50	52	54	
56	58	60	62	

64	66	68	70	
72	74	76	78	
80	82	84	86	
88	90	92	94	

96	98	100	102	
104	106	108	110	
112	114	116	118	
120	122	124	126	

valeur de x : 4 , valeur de y : 4 
valeur de x : 5 , valeur de y : 12 
valeur de x : 6 , valeur de y : 20 
valeur de x : 7 , valeur de y : 28 
Matrix c :
112	136	136	160	
304	392	392	480	
496	648	648	800	
688	904	904	1120	

880	1160	1160	1440	
1072	1416	1416	1760	
1264	1672	1672	2080	
1456	1928	1928	2400	

1648	2184	2184	2720	
1840	2440	2440	3040	
2032	2696	2696	3360	
2224	2952	2952	3680	

2416	3208	3208	4000	
2608	3464	3464	4320	
2800	3720	3720	4640	
2992	3976	3976	4960	

Matrix c :
112	124	136	148	
304	348	392	436	
496	572	648	724	
688	796	904	101